# PatchTST for the forecast of German Energy consumption

TODO
- Import necessary code
- See Nick's structure and write a similar notebook with similar sections.
- Check in original PatchTST repo code where are the preds
- check to see if I can just discard the first 56
- run my model with my dataset and with regular dataset but discarding 56 outputs and see which is more accurate
- If I can't find it, just run the model with my code, verify it works (how?) and stick to that way of windowing
- Create new dataset class for the supermarket data (gap?)
- Order everything in this notebook